In [1]:
import pandas as pd
import numpy as np
import datetime
from itertools import product

**Must haves for this notebook to work**
* yTrain and yTest files
* columns labeled EquipmentID, EventTimeStamp, target, prediction for both files
* target should be a column of 1's and 0's
* prediction should be a numerical value between 1 and 0

*Once above is done then you need to move your training file and test file into the data folder within the folder the notebook is in.*

*Finally replace 'YOUR_X_FILE_NAME_HERE' with training in first cell and test in the second. Keep the '' around your file name*

*Once done you can run all cells.*

**Training read in**

In [2]:
yTrain = pd.read_csv('../data/{file_name}.csv'.format(file_name = 'hg_yTrain_full'))
yTrain

,target,EquipmentID,EventTimeStamp,prediction
0,0.0,302,2010-12-31 23:04:15,0.012843
1,0.0,302,2015-02-21 12:52:42,0.006960
2,0.0,302,2015-02-21 12:52:42,0.005681
3,0.0,302,2015-02-21 15:40:57,0.004487
4,0.0,302,2015-02-21 15:40:57,0.003674
...,...,...,...,...
433399,0.0,2377,2020-02-12 05:42:02,0.004857
433400,0.0,2377,2020-02-28 04:56:18,0.009077
433401,0.0,2377,2020-03-03 07:47:01,0.000911
433402,0.0,2377,2020-03-06 14:14:13,0.005996


**Test read in**

In [3]:
yTest = pd.read_csv('../data/{file_name}.csv'.format(file_name = 'hg_yTest_full'))
yTest

,target,EquipmentID,EventTimeStamp,prediction
0,0.0,301,2015-05-11 13:11:20,0.006708
1,0.0,301,2015-05-13 08:22:32,0.003864
2,0.0,301,2015-05-18 09:34:05,0.062225
3,0.0,301,2015-05-21 13:57:35,0.001435
4,0.0,301,2015-05-28 13:31:41,0.036742
...,...,...,...,...
113011,0.0,2374,2020-01-25 09:17:42,0.001410
113012,0.0,2374,2020-01-31 12:40:07,0.000071
113013,0.0,2374,2020-02-24 13:37:07,0.000095
113014,0.0,2374,2020-02-27 15:40:00,0.000842


*Reading in data of derates that will be used to make a list. Will already be in the data folder.*

In [4]:
derates = pd.read_csv('../data/derates.csv').drop_duplicates()

*Making sure all time columns are a datetime.*

In [5]:
derates['EventTimeStamp'] = pd.to_datetime(derates['EventTimeStamp'])
yTrain['EventTimeStamp'] = pd.to_datetime(yTrain['EventTimeStamp'])
yTest['EventTimeStamp'] = pd.to_datetime(yTest['EventTimeStamp'])

In [6]:
derates = derates.loc[(derates.groupby('EquipmentID').diff().fillna(pd.to_timedelta(10,'d')) >= '7 D')['EventTimeStamp']]

In [7]:
derates.shape

(296, 2)

*The nasty function. Will try to explain what happens in comments if you are interested*

In [8]:
def model_eval(yTrain, yTest, derates = derates):
    #Creating a unique list of equipment ids that have had a derate.
    derateTrucks = derates['EquipmentID'].drop_duplicates().to_list()
    
    #Using the list made above to create a list of all the trucks that have had a derate in the training data. 
    trainDeratesTrucks = [x for x in derateTrucks if x in yTrain['EquipmentID'].to_list()]
    
    #The first for loop is removing rows around a derate so false postives can be calculated
    removal = []
    for truck in trainDeratesTrucks:
        dates = (
            derates
            .loc[derates['EquipmentID'] == truck]
            #.sort_values(by = 'EventTimeStamp')   #First chunk is going through and grabbing derate times by equipment id
            #.drop_duplicates('EquipmentID')
            .reset_index(drop = True)['EventTimeStamp']
        )
        for date in dates:
            pre = date - datetime.timedelta(hours = 24 * 7)   #calculating a week before and week after a derate
            post = date + datetime.timedelta(hours = 24 * 7)

            times = yTrain.loc[                      
                (yTrain['EquipmentID'] == truck) &     #Using those times above to filter yTrain 
                (yTrain['EventTimeStamp'] >= pre) &
                (yTrain['EventTimeStamp'] <= post)
            ].reset_index(drop = True)

            removal.append(times)    #appending to empty list so it can be turned into a dataframe
            
    removal = (
        yTrain
        .merge(
            pd.concat(removal),       #"Anti-merging" the dataframe we created above so we can remove those values
            how = 'outer', 
            on = ['EquipmentID', 'EventTimeStamp', 'target', 'prediction'], 
            indicator = True
        )
    )

    yTrainNoDerate = removal.loc[removal['_merge'] == 'left_only'] #finalizing the "anti-merge"
    
    ######################################################################################################################
    
    #These are the different threshold values I looped through. Feel free to change them if you see something.
    varience = [.99, .95, .9, .85, .8, .75, .7, .6, .5, .4, .3]
    hoursPre = [5, 4, 3 , 2, 1]

    #This for loop calculates the entire confusion matrix on the training data at different thresholds
    df = []
    for x in product(varience, hoursPre):      #This is looping through the list we created above
        truePos = []
        for truck in trainDeratesTrucks:
            dates = (
                derates
                .loc[derates['EquipmentID'] == truck]     #Same as the loop above this is grabbing times of derates
                #.sort_values(by = 'EventTimeStamp')
                #.drop_duplicates('EquipmentID')
                .reset_index(drop = True)['EventTimeStamp']
            )
            for date in dates:
                pre = date - datetime.timedelta(hours = 24) #Pre is the same as above
                post = date - datetime.timedelta(hours = x[1])  #Post is now 1 hour prior to a derate 

                times = yTrain.loc[
                    (yTrain['EquipmentID'] == truck) &
                    (yTrain['EventTimeStamp'] >= pre) &  #Using those times above to filter yTrain 
                    (yTrain['EventTimeStamp'] <= post)
                ].reset_index(drop = True)
                
                #searching to see if the max prediciton is greater than or equal to the set threshold
                truePos.append(int(times['prediction'].max() >= x[0])) #True Positive
        
        #Summing the times there was more than a day between when the model predicited there was a derate when there wasn't a derate
        falsePos = (                        #False Positive
            yTrainNoDerate
            .loc[yTrainNoDerate['prediction'] > x[0]]
            .groupby('EquipmentID')['EventTimeStamp']
            .diff()
            .fillna(pd.to_timedelta(5,'d')) > '1 D'
        ).sum()

        df.append({
            'threshold' : x[0],
            'hoursPre' : x[1],
            'truePos' : sum(truePos),
            'falseNeg' : len(truePos) - sum(truePos),  #Building the dataframe confusion matrix
            'falsePos' : falsePos,
            'trueNeg' : len(yTrainNoDerate) - falsePos
        })
        
    confusionMatrix = pd.DataFrame(df)
    #calculated the total gained or lost by our model
    confusionMatrix['netP_L'] = confusionMatrix['truePos'] * 4000 - confusionMatrix['falsePos'] * 500
    #ordering by best threshold to worst
    confusionMatrix = confusionMatrix.sort_values('netP_L', ascending = False).reset_index(drop = True)
    
    #####################################################################################################################
    
    #below are the same steps above but for the testing data and only using the best threshold that was found in the 
    #training data
    
    testDeratesTrucks = [x for x in derateTrucks if x in yTest['EquipmentID'].to_list()]
    removalTest = []
    for truck in testDeratesTrucks:
        dates = (
            derates
            .loc[derates['EquipmentID'] == truck]
            #.sort_values(by = 'EventTimeStamp')
            #.drop_duplicates('EquipmentID')
            .reset_index(drop = True)['EventTimeStamp']
        )
        for date in dates:
            pre = date - datetime.timedelta(hours = 24 * 7)
            post = date + datetime.timedelta(hours = 24 * 7)

            times = yTest.loc[
                (yTest['EquipmentID'] == truck) &
                (yTest['EventTimeStamp'] >= pre) &
                (yTest['EventTimeStamp'] <= post)
            ].reset_index(drop = True)

            removalTest.append(times)
            
    removalTest = (
        yTest
        .merge(
            pd.concat(removalTest), 
            how = 'outer', 
            on = ['EquipmentID', 'EventTimeStamp', 'target', 'prediction'], 
            indicator = True
        )
    )

    yTestNoDerate = removalTest.loc[removalTest['_merge'] == 'left_only']
    
    ####################################################################################################################
    
    testDeratesTrucks = [x for x in derateTrucks if x in yTest['EquipmentID'].to_list()]
    dfTest = []
    truePos = []
    for truck in testDeratesTrucks:
        dates = (
            derates
            .loc[derates['EquipmentID'] == truck]
            #.sort_values(by = 'EventTimeStamp')
            #.drop_duplicates('EquipmentID')
            .reset_index(drop = True)['EventTimeStamp']
        )
        for date in dates:
            pre = date - datetime.timedelta(hours = 24)
            post = date - datetime.timedelta(hours = int(confusionMatrix['hoursPre'][0]))

            times = yTest.loc[
                (yTest['EquipmentID'] == truck) &
                (yTest['EventTimeStamp'] >= pre) &
                (yTest['EventTimeStamp'] <= post)
            ].reset_index(drop = True)

            truePos.append(int(times['prediction'].max() >= confusionMatrix['threshold'][0]))

    falsePos = (
        yTestNoDerate
        .loc[yTestNoDerate['prediction'] > confusionMatrix['threshold'][0]]
        .groupby('EquipmentID')['EventTimeStamp']
        .diff()
        .fillna(pd.to_timedelta(5,'d')) > '1 D'
    ).sum()

    dfTest.append({
        'var' : confusionMatrix['threshold'][0],
        'hoursPre' : confusionMatrix['hoursPre'][0],
        'truePos' : sum(truePos),
        'falseNeg' : len(truePos) - sum(truePos),
        'falsePos' : falsePos,
        'trueNeg' : len(yTestNoDerate) - falsePos
    })
    
    confusionMatrixTest = pd.DataFrame(dfTest)
    confusionMatrixTest['netP_L'] = confusionMatrixTest['truePos'] * 4000 - confusionMatrixTest['falsePos'] * 500
    
    return confusionMatrix, confusionMatrixTest

In [9]:
confusionMatrixTrain = model_eval(yTrain, yTest)[0]

confusionMatrixTrain

,threshold,hoursPre,truePos,falseNeg,falsePos,trueNeg,netP_L
0,0.95,1,117,124,365,426251,285500
1,0.90,1,129,112,488,426128,272000
2,0.85,1,142,99,617,425999,259500
3,0.95,2,110,131,365,426251,257500
4,0.90,2,122,119,488,426128,244000
5,0.99,1,80,161,156,426460,242000
6,0.85,2,135,106,617,425999,231500
7,0.80,1,147,94,714,425902,231000
8,0.95,3,103,138,365,426251,229500
9,0.90,3,115,126,488,426128,216000


In [10]:
confusionMatrixTest = model_eval(yTrain, yTest)[1]

confusionMatrixTest

,var,hoursPre,truePos,falseNeg,falsePos,trueNeg,netP_L
0,0.95,1,24,31,149,111648,21500
